In [1]:
from numpy import sin, cos, random as rn, array

In [2]:
'''
Defined Function
'''
def h(x1, x2):
    assert -1 <= x1 <= 2, 'Outside defined limit for x1!'
    assert -1 <= x2 <= 1, 'Outside defined limit for x2!'
    return cos(x1) * sin(x2) - x1 / (x2 ** 2 + 1)

In [72]:
'''
Used to generate a phenotype based on generated genotype
'''
def x(r_min, r_max, gens):
    assert r_min <= r_max, 'Minimum limit must be lesser than maximum limit!'
    g_sum = sum(gens)
    N = len(gens)
    return r_min + (r_max - r_min) * g_sum / N

In [4]:
'''
Generate genotype based on random real number
'''
def generate_genotype(m, n):
    assert m >= 0, 'Accepts positive integers!'
    assert m % 2 == 0, 'Accepts even integers!'
    return [[rn.random() for i in range(m)] for i in range(2**n)]

In [5]:
'''
Generate list of phenotype based on genotype given
'''
def generate_phenotype(gene):
    N = len(gene)
    x1 = x(-1, 2, gene[0:N//2])
    x2 = x(-1, 1, gene[N//2:N])
    return x1, x2

In [6]:
'''
Returns minimum fitness value. If x1 and x2 is outside boundary,
then it will return 0 and determined formula if it is within.
'''
def fitness(x1, x2):
    try:
        return -h(x1, x2)
    except:
        return 0

In [7]:
'''
Calculate fitness of a gene
'''
def calculate_gene_fitness(gene):
    x1, x2 = generate_phenotype(gene)
    return fitness(x1, x2)

In [8]:
'''
Evaluate fitness value of entire genes
'''
def evaluate_genes(genes):
    phenotypes = [generate_phenotype(gene) for gene in genes]
    genes_fitness = [fitness(x1, x2) for x1, x2 in phenotypes]
    evaluated_result = sum(genes_fitness) 
    return evaluated_result

In [9]:
'''
Sort genes based on its fitness value
'''
def sort_genes(genes):
    return sorted(genes, key=lambda gene: calculate_gene_fitness(gene))

In [76]:
'''
Tournament selection for selecting best parents,
it will do k iteration to find values that is
suitable based on its fitness value
'''
def selection(genes, k):
    copied_genes = genes.copy()
    best = []
    best_fitness = 0
    for i in range(k):
        random_index = rn.randint(len(copied_genes))
        indv_gene = copied_genes[random_index]
        indv_fitness = calculate_gene_fitness(indv_gene)
        if len(best) == 0 or indv_fitness > best_fitness:
            copied_genes.pop(random_index)
            best.append(indv_gene)
            best_fitness = indv_fitness
    return best

In [11]:
'''
Crossover for genes, by default has probability of 65%
'''
def crossover(genes, pc):
    N = len(genes[0])
    crossed_genes = []
    while len(crossed_genes) < len(genes):
        cross_probability = rn.randint(0, 101)
        i = len(crossed_genes)
        new_gene = genes[i].copy()
        random_i = rn.randint(len(genes))
        if cross_probability <= pc and random_i != i:            
            random_gene = genes[random_i].copy()
            new_gene[0:N//2-1], random_gene[0:N//2+1] = random_gene[N//2+1:N], new_gene[N//2-1:N]
        crossed_genes.append(new_gene)
    return crossed_genes

In [12]:
'''
Mutation for genes, by default has probability of 1%
'''
def mutation(genes, pm):
    mutated_genes = []
    N = len(genes[0])
    for i in range(len(genes)):
        mutation_probability = rn.randint(0, 101)
        gene = genes[i].copy()
        if mutation_probability <= pm:
            random_i = rn.randint(N)
            gene[random_i] = rn.random()
        mutated_genes.append(gene)
    return mutated_genes

In [287]:
def elitism(genes, k):
    sorted_genes = sort_genes(genes)
    return genes[-k:]

In [363]:
def generational_replacement_ga(k, m, n, o, pc, pm):
    N = 2 ** n
    genes = generate_genotype(m, n)
    for i in range(N):
        new_genes = elitism(genes, 2)
        while len(new_genes) < N:
            parents = selection(genes, k)
            offspring = crossover(parents, pc)
            offspring = mutation(offspring, pm)
            new_genes.extend(offspring)
        genes = new_genes
    best = max(genes, key=lambda gene: calculate_gene_fitness(gene))
    x1, x2 = generate_phenotype(best)
    return x1, x2, h(x1, x2)

generational_replacement_ga(10, 6, 4, 2, 65, 1)

(1.4144623353167587, 0.06740915010652171, -1.3975765807274574)

In [64]:
'''
Steady state process for survivor selection
'''
def steady_state(genes, offspring):
    N = len(offspring)
    worst_genes = sort_genes(genes)[N:]
    replaced_index = []
    for i in range(N):
        genes.remove(worst_genes[i])
        genes.append(offspring[i])
    return genes

In [366]:
def steady_state_ga(k = 10, m = 6, n = 4):
    genes = generate_genotype(m, n)
    for i in range(2**n):
        evaluated_fitness = evaluate_genes(genes)
        parents = selection(genes, k)
        offspring = crossover(parents, 65)
        offspring = mutation(parents, 1)
        genes = sort_genes(genes)
        genes = steady_state(genes, offspring)
    best = max(genes, key=lambda gene: calculate_gene_fitness(gene))
    x1, x2 = generate_phenotype(best)
    return x1, x2, h(x1, x2)

steady_state_ga(10, 6, 4)

(1.585985657031718, 0.21819598451408773, -1.517197103614921)